In [11]:
import requests
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd
import re
import datetime 
from konlpy.tag import Hannanum
from config import * # config안의 정보 가져오기
from transformers import pipeline
rc('font', family='AppleGothic')

plt.rcParams['axes.unicode_minus'] = False

In [12]:
def resp_data (url, query, headers, ELASTIC_SEARCH_AUTH, type):
    '''type for POST, GET ... '''

    resp = requests.get(
        url,
        data=query.encode('utf-8'),
        headers=headers,
        auth = ELASTIC_SEARCH_AUTH
    )

    order = type

    return requests.order(url, data=query, headers=headers, auth = ELASTIC_SEARCH_AUTH).json()

In [13]:
def get_today_stock_news_data_list (ticker_name):

    #start_time = datetime.datetime.today().strftime("%Y-%m-%dT00:00:00")
    #end_time = datetime.datetime.today().strftime("%Y-%m-%dT23:59:59") 이건 오늘 날짜로 따질때 이야기

    start_time = "2022-04-05T00:00:00"
    end_time = "2022-04-05T23:59:59"


    headers = {
    'Content-Type': 'application/json'
    }

    url = f"{ELASTIC_SEARCH_URL}/news/_count"

    query=f'''{{
    "query": {{
      "bool": {{
        "must": [
          {{
            "match": {{
              "body": "{ticker_name}"
            }}
          }},
          {{
            "range": {{
              "created_at": {{
               "gte": "{start_time}",
               "lte": "{end_time}"
              }}
            }}
          }}
        ]
      }}
    }}
    }}'''

    count = int(requests.get(url, data=query.encode('utf-8'), headers=headers, auth = ELASTIC_SEARCH_AUTH).json()['count'])

    if count != 0:

      url = f"{ELASTIC_SEARCH_URL}/news/_search"

      query=f'''{{
      "size": {count},
      "query": {{
        "bool": {{
          "must": [
            {{
              "match": {{
                "body": "{ticker_name}"
              }}
            }},
            {{
              "range": {{
                "created_at": {{
                "gte": "{start_time}",
                "lte": "{end_time}"
                }}
              }}
            }}
          ]
        }}
      }}
      }}'''  

      resp = requests.get(url, data=query.encode('utf-8'), headers=headers, auth = ELASTIC_SEARCH_AUTH).json()

      resp = resp['hits']['hits']
    
    else:

      resp = []


    return resp



In [14]:
def clean_doc (doc):
    
    doc = re.sub(r'[^\wㄱ-ㅎ가-힣]', ' ', doc)
    doc = re.sub('  ', ' ', doc)
    doc = re.sub('  ', ' ', doc)

    return doc

In [15]:
def title_extract(doc):
    
    return doc['_source']['title']

def body_extract (doc):

    return doc['_source']['body']

In [16]:
stock1_doc_list = get_today_stock_news_data_list("삼성전자")

In [17]:
title_list = [title_extract(x) for x in stock1_doc_list]
#title_list = [clean_doc(x) for x in title_list]
title_list[0:10]

['3년 전 산 삼성전자 지분이 지금은… 블랙록의 돌부처 투자법 [왕개미연구소]',
 "상속세 때문에 홍라희가 판 삼성전자 지분, '이곳'이 가져갔다",
 '임직원과 소통 나선 한종희, 부회장 된 후 첫 이메일 보낸 사연',
 "'JH 입니다' 한종희 삼성 부회장, 임직원과 이메일 소통 나섰다",
 "삼성전자 부회장 'JH'의 소통…임직원 건의 직접 듣고 바꿨다",
 "삼성 'JH' 한종희 부회장, 임직원에 이메일 보내 '소통 행보'",
 "JP모간, 홍라희 삼전 지분 매입뒤 1900건 넘게 거래…또 '단타'?",
 '코스피, 강보합 마감…2760선 턱밑',
 '반도체 업계 또 놀라겠네…DB하이텍 파격 보상 "신입 연봉 7200만원"',
 "삼성 한종희 부회장 '소통행보'…건의사항에 이메일로 화답"]

### 번역기 돌리기

In [18]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-ko-en")

In [19]:
title_list[0:10]

['3년 전 산 삼성전자 지분이 지금은… 블랙록의 돌부처 투자법 [왕개미연구소]',
 "상속세 때문에 홍라희가 판 삼성전자 지분, '이곳'이 가져갔다",
 '임직원과 소통 나선 한종희, 부회장 된 후 첫 이메일 보낸 사연',
 "'JH 입니다' 한종희 삼성 부회장, 임직원과 이메일 소통 나섰다",
 "삼성전자 부회장 'JH'의 소통…임직원 건의 직접 듣고 바꿨다",
 "삼성 'JH' 한종희 부회장, 임직원에 이메일 보내 '소통 행보'",
 "JP모간, 홍라희 삼전 지분 매입뒤 1900건 넘게 거래…또 '단타'?",
 '코스피, 강보합 마감…2760선 턱밑',
 '반도체 업계 또 놀라겠네…DB하이텍 파격 보상 "신입 연봉 7200만원"',
 "삼성 한종희 부회장 '소통행보'…건의사항에 이메일로 화답"]

In [20]:
[translator(x) for x in title_list[0:10]]

[[{'translation_text': "Three years ago, you've got a substrate... now... of Black Rock's breakthrough investment."}],
 [{'translation_text': "Because of the inheritance tax, Hong Lahee's subdivision, this place, was taken from him."}],
 [{'translation_text': "I've been communicating with my board of directors. I've been sending you my first email after becoming vice president."}],
 [{'translation_text': "It's JH."}],
 [{'translation_text': 'I heard and switched directly to JH, vice president of the Triads.'}],
 [{'translation_text': 'Samsung JH, VP, send an e-mail to your boardman.'}],
 [{'translation_text': 'JP Morgan, Hong-ray, three-year-old purchase, more than 1,900 trades... and Danta?'}],
 [{'translation_text': 'Cospi, end of supple... 2760-line jaws.'}],
 [{'translation_text': "I'm gonna be surprised again by the semiconductor industry... to reward the DB High Tech break."}],
 [{'translation_text': "I e-mailed an e-mail to the sub-station's vice president, Sassung Jong-hee."}]]

In [28]:
from googletrans import Translator

sentiment_analysis = pipeline("sentiment-analysis", model="siebert/sentiment-roberta-large-english")
translator = Translator()


Downloading:   0%|          | 0.00/687 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [22]:
title_list[0]

'3년 전 산 삼성전자 지분이 지금은… 블랙록의 돌부처 투자법 [왕개미연구소]'

In [47]:
!pip install finance-datareader

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached finance_datareader-0.9.33-py3-none-any.whl (48 kB)
  Using cached requests_file-1.5.1-py2.py3-none-any.whl (3.7 kB)


In [48]:
import FinanceDataReader as fdr

In [25]:
doc = [translator.translate(x, dest='en') for x in title_list[0:10]]

print(doc)

[<googletrans.models.Translated object at 0x2a072baf0>, <googletrans.models.Translated object at 0x29d77f9a0>, <googletrans.models.Translated object at 0x29d77ec70>, <googletrans.models.Translated object at 0x29d765f40>, <googletrans.models.Translated object at 0x29d768a90>, <googletrans.models.Translated object at 0x2a06f7fa0>, <googletrans.models.Translated object at 0x2a0706be0>, <googletrans.models.Translated object at 0x2a071b940>, <googletrans.models.Translated object at 0x29d772fa0>, <googletrans.models.Translated object at 0x2a0704b20>]


In [37]:
a = sentiment_analysis("Because of inheritance tax, Rahee Hong sold Samsung Electronics stake, 'here' took it")[0]
a['label']
a['score']

0.9540044665336609

In [44]:
doc

In [46]:
for i, j in zip(doc, title_list[0:10]):
    a = sentiment_analysis(i.text)[0]
    print(i.text)
    print(j)
    print(f'''위 기사는 {round(a['score'], 4) * 100}% 확률로 "{a['label']}"인 기사입니다.''' + '\n')

The stake in Samsung Electronics, which I bought three years ago, is now… Black Rock's Stone Buddha Investment Method [King Ant Research Institute]
3년 전 산 삼성전자 지분이 지금은… 블랙록의 돌부처 투자법 [왕개미연구소]
위 기사는 99.22999999999999% 확률로 "POSITIVE"인 기사입니다.

Because of inheritance tax, Rahee Hong sold Samsung Electronics stake, 'here' took it
상속세 때문에 홍라희가 판 삼성전자 지분, '이곳'이 가져갔다
위 기사는 95.39999999999999% 확률로 "POSITIVE"인 기사입니다.

The story of Jong-hee Han, who communicated with executives and employees, sent the first email after becoming vice-chairman
임직원과 소통 나선 한종희, 부회장 된 후 첫 이메일 보낸 사연
위 기사는 99.82% 확률로 "POSITIVE"인 기사입니다.

'This is JH' Samsung Vice Chairman Han Jong-hee, e-mail communication with executives and employees
'JH 입니다' 한종희 삼성 부회장, 임직원과 이메일 소통 나섰다
위 기사는 85.46000000000001% 확률로 "POSITIVE"인 기사입니다.

Communication of Samsung Electronics Vice Chairman 'JH'... Directly listened to employee suggestions and changed
삼성전자 부회장 'JH'의 소통…임직원 건의 직접 듣고 바꿨다
위 기사는 99.78% 확률로 "POSITIVE"인 기사입니다.

Samsung 'JH' Jong-hee